# Get Minute Trend Data from the LIGO Sites

In [1]:
# Library Imports and Python parameter settings
%matplotlib inline

from __future__ import division
import numpy as np
#import matplotlib.pyplot as plt
import scipy.io as sio
#import scipy.signal as sig
#import scipy.constants as const
from astropy.time import Time
#import sys
#sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7')
import nds2

## setup the servers, start times, and duration

In [3]:
ifo = 'H1'
# Setup connection to the NDS
conn = nds2.connection('nds.ligo.caltech.edu', 31200)

# Setup start and stop times
times = '2017-03-01 00:00:00'
t = Time(times, format='iso', scale='utc')
#t_start = int(t.gps)
t_start = int(np.floor(t.gps/60)*60) # round start time to multiple of 60 for minute trend
dur_in_days = 1
dur_in_minutes = dur_in_days * 24 * 60
dur = dur_in_minutes * 60    # must be a multiple of 60

## Build up the channel list and Get the Data

In [4]:
chan_head = ifo + ':' + 'ISI-' + 'GND_STS' + '_'
sensors = {'HAM2'}
dofs = {'X', 'Y', 'Z'}
bands = {'30M_100M', '100M_300M', '300M_1', '1_3', '3_10', '10_30'}
channels = []
# why is the channel ordering so weird? 
# need to use sorted to preserve the intended ordering
for sensor in sorted(sensors):
    for dof in sorted(dofs):
        for band in sorted(bands):
            channel = chan_head + sensor + '_' + dof + '_BLRMS_' + band + '.mean, m-trend'
            #print channel
            channels.append(channel)

data = conn.fetch(t_start, t_start + dur, channels)

for i in channels:
    print(i)

H1:ISI-GND_STS_HAM2_X_BLRMS_100M_300M.mean, m-trend
H1:ISI-GND_STS_HAM2_X_BLRMS_10_30.mean, m-trend
H1:ISI-GND_STS_HAM2_X_BLRMS_1_3.mean, m-trend
H1:ISI-GND_STS_HAM2_X_BLRMS_300M_1.mean, m-trend
H1:ISI-GND_STS_HAM2_X_BLRMS_30M_100M.mean, m-trend
H1:ISI-GND_STS_HAM2_X_BLRMS_3_10.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_100M_300M.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_10_30.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_1_3.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_300M_1.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_30M_100M.mean, m-trend
H1:ISI-GND_STS_HAM2_Y_BLRMS_3_10.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_100M_300M.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_10_30.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_1_3.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_300M_1.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_30M_100M.mean, m-trend
H1:ISI-GND_STS_HAM2_Z_BLRMS_3_10.mean, m-trend


## save the data so that it can be loaded by matlab or python
#### savemat will compress the data and save it in hdf5 format

In [8]:
vdata = []
# get the data and stack it into a single matrix where the data are the columns
for k in range(len(channels)):
    vdata.append(data[k].data)

# save to a hdf5 format that matlab can read (why is compression off by default?)
sio.savemat(ifo + '_SeismicBLRMS.mat', mdict={'data': vdata}, do_compression=True)

### some debugging info about the channels

In [9]:
print("Channel name is " + data[0].channel.name)
print("Sample rate is " + str(data[0].channel.sample_rate) + " Hz")
print("Number of samples is " + str(data[0].length))
print("GPS Start time is " + str(data[0].gps_seconds))

Channel name is H1:ISI-GND_STS_HAM2_X_BLRMS_100M_300M.mean
Sample rate is 0.0166666992009 Hz
Number of samples is 1440
GPS Start time is 1172361600


In [ ]:
dir(data[0])

In [ ]:
dir(data[0].channel)

In [ ]:
conn.fetch(t_start, t_start + dur, ['H1:ISI-GND_STS_HAM2_Y_BLRMS_300M_1.mean,m-trend'])

In [ ]:
channels

In [ ]:
print t_start
print int(np.floor(t_start/60)*60)